# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [4]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [57]:
# file location of csv
csvpath = os.path.join('..','WeatherPy','cities.csv')

# read csv into dataframe
cities_pd = pd.read_csv(csvpath)

cities_pd.head()

,Unnamed: 0,City,Lat,Lon,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Atuona,-9.8000,-139.0333,79.63,78,81,23.42,PF,1612681448
1,1,Distrikt Brokopondo,4.7500,-55.1667,74.95,90,100,7.94,SR,1612681561
2,2,Punta Arenas,-53.1500,-70.9167,46.40,81,0,10.36,CL,1612681016
3,3,Maumere,-8.6199,122.2111,83.03,75,97,6.11,ID,1612681561
4,4,Airai,-8.9266,125.4092,68.79,98,100,3.27,TL,1612681315


In [70]:
cities_pd.iloc[:,0]

0        0
1        1
2        2
3        3
4        4
      ... 
543    543
544    544
545    545
546    546
547    547
Name: Unnamed: 0, Length: 548, dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [29]:
# configure gmaps
gmaps.configure(api_key=g_key)

# use lat & long as locations
# Store 'Lat' and 'Lng' into  locations 
locations = cities_pd[["Lat", "Lon"]].astype(float)
humidity = cities_pd['Humidity'].tolist()
len(humidity)

548

In [28]:
# Create a poverty Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [47]:
ideal_cities =cities_pd[(cities_pd['Max Temp (F)'] > 70) & 
                      (cities_pd['Max Temp (F)'] < 80) & 
                      (cities_pd['Wind Speed'] < 10) &
                      (cities_pd['Cloudiness'] == 0)]
len(ideal_cities)
ideal_cities

,Unnamed: 0,City,Lat,Lon,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date
7,7,Mataura,-46.1927,168.8643,70.27,66,0,3.56,NZ,1612681445
19,19,Kruisfontein,-34.0033,24.7314,77.00,64,0,7.00,ZA,1612681274
70,70,Devgarh,25.5333,73.9000,72.39,21,0,6.78,IN,1612681569
223,223,Gore,-46.1028,168.9436,70.14,71,0,3.42,NZ,1612681589
230,230,Laï,9.3952,16.2989,72.37,21,0,4.18,TD,1612681511
240,240,Biaora,23.8667,76.9167,74.86,20,0,4.63,IN,1612681591
244,244,Road Town,18.4167,-64.6167,78.01,90,0,5.99,VG,1612681226
253,253,Vao,-22.6667,167.4833,78.66,81,0,9.06,NC,1612681593
265,265,Willowmore,-33.2926,23.4895,70.25,46,0,4.14,ZA,1612681594
306,306,Berbérati,4.2612,15.7922,79.47,56,0,1.43,CF,1612681599


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [78]:
hotel_df = ideal_cities[['City','Lat','Lon','Max Temp (F)','Humidity','Cloudiness','Wind Speed','Country','Date']]
hotel_df['Hotel Name'] = ''
hotel_df

/Users/Howard/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Lat,Lon,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
7,Mataura,-46.1927,168.8643,70.27,66,0,3.56,NZ,1612681445,
19,Kruisfontein,-34.0033,24.7314,77.00,64,0,7.00,ZA,1612681274,
70,Devgarh,25.5333,73.9000,72.39,21,0,6.78,IN,1612681569,
223,Gore,-46.1028,168.9436,70.14,71,0,3.42,NZ,1612681589,
230,Laï,9.3952,16.2989,72.37,21,0,4.18,TD,1612681511,
240,Biaora,23.8667,76.9167,74.86,20,0,4.63,IN,1612681591,
244,Road Town,18.4167,-64.6167,78.01,90,0,5.99,VG,1612681226,
253,Vao,-22.6667,167.4833,78.66,81,0,9.06,NC,1612681593,
265,Willowmore,-33.2926,23.4895,70.25,46,0,4.14,ZA,1612681594,
306,Berbérati,4.2612,15.7922,79.47,56,0,1.43,CF,1612681599,


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
